The data is downloaded from [Qiita study 2538](https://qiita.ucsd.edu/study/description/2538):
    
- `mapping-file.txt` is downloaded from the only 16S artifact.
- `otu_table.biom`, note these OTUs are from sequences trimmed at 100 nucleotides. Picked closed reference 97%.

Let's get rid of blank samples, we don't use them for any of the analyses.

In [2]:
!filter_samples_from_otu_table.py -i otu_table.biom \
-m mapping-file.txt \
-s 'sample_type:stool' \
-o otu-table.noblanks.biom

Calculate alpha diversity for a couple metrics and then we'll add this to the mapping file

In [3]:
mkdir -p analysis/alpha

In [5]:
!echo "make_rarefaction_plots:colorby illumina_technology" > analysis/alpha-params.txt
!echo "alpha_diversity:metrics shannon,PD_whole_tree,chao1,observed_species" >> analysis/alpha-params.txt

In [6]:
!alpha_rarefaction.py \
-i otu-table.noblanks.biom \
-m mapping-file.txt \
-o analysis/alpha \
-n 10 \
--min_rare_depth 500 \
-e 12000 \
-t gg-tree/97_otus.tree \
--retain_intermediate_files \
-p analysis/alpha-params.txt -f

In [7]:
!add_alpha_to_mapping_file.py \
-i analysis/alpha/alpha_div_collated/PD_whole_tree.txt,\
analysis/alpha/alpha_div_collated/shannon.txt,\
analysis/alpha/alpha_div_collated/chao1.txt,\
analysis/alpha/alpha_div_collated/observed_species.txt \
-m mapping-file.txt \
--depth 7400 \
--collated_input \
--binning_method quantile \
-x NA \
-b 4 \
-o mapping-file.alpha.txt

In [10]:
cp analysis/alpha/rarefaction/rarefaction_7400_0.biom otu-table.noblanks.even.7400.biom

In [11]:
!echo "make_emperor:verbose True" > analysis/beta-params.txt

In [12]:
!beta_diversity_through_plots.py \
-i otu-table.noblanks.biom \
-m mapping-file.alpha.txt \
-o analysis/beta \
-e 7400 \
-t gg-tree/97_otus.tree \
-p analysis/beta-params.txt
#-a -O 4 -f

/Users/yoshikivazquezbaeza/miniconda/envs/qiime-191/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -2.14739103621 and the largest is 55.2500741976.
  RuntimeWarning
/Users/yoshikivazquezbaeza/miniconda/envs/qiime-191/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the N